<a href="https://colab.research.google.com/github/egonik-unlp/tp_curso_ml/blob/main/src/notebooks/nn_transmembrane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/egonik-unlp/tp_curso_ml.git
!git pull
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from google.colab import drive
from tensorflow.keras.utils import to_categorical
drive.mount('/content/drive')
import os
from datetime import datetime

Already up to date.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/tp_curso_ml/src/pickle/numpy_arrays/
data=np.load('data_compressed.npz', allow_pickle=True)

/content/tp_curso_ml/src/pickle/numpy_arrays


In [ ]:
X_1=data.f.seqs
labels_0=data.f.labels_tmd.reshape(-1,1)
labels_1=data.f.labels_ntmd.reshape(-1,1)

In [ ]:
def to_categorial_transmembrane(*arrays):
  retval=[]
  for array in arrays:
    retval.append(keras.utils.to_categorical(array))
  return retval

In [ ]:
#Labels_0

X_train_val, X_test, t_train_val, t_test_0=train_test_split(X_1, labels_0, random_state=42)
X_train,X_val,t_train_0,t_val_0=train_test_split(X_train_val, t_train_val, random_state=42)

#Labels_1

X_train_val, X_test, t_train_val, t_test_1=train_test_split(X_1, labels_1, random_state=42)
X_train,X_val,t_train_1,t_val_1=train_test_split(X_train_val, t_train_val, random_state=42)

t_train_1_ohe=to_categorical(t_train_1)
t_val_1_ohe=to_categorical(t_val_1)
t_test_1_ohe=to_categorical(t_test_1)

In [ ]:
tf.keras.backend.clear_session()
input_=keras.layers.Input(shape=(2000,1))
conv1=keras.layers.Conv1D(filters=32, kernel_size=3,activation='relu')(input_)
conv2=keras.layers.Conv1D(filters=32, kernel_size=3,activation='relu')(conv1)
pool1=keras.layers.MaxPooling1D(pool_size=4)(conv2)
conv3=keras.layers.Conv1D(filters=64, kernel_size=4,activation='relu')(pool1)
conv4=keras.layers.Conv1D(filters=64, kernel_size=4,activation='relu')(conv3)
pool2=keras.layers.MaxPooling1D(pool_size=4)(conv4)
flatten=keras.layers.Flatten()(pool2)
dense1=keras.layers.Dense(128, activation='relu')(flatten)
output_=keras.layers.Dense(1, activation='sigmoid')(dense1)
model=keras.models.Model(inputs=[input_], outputs=[output_])

In [ ]:
folder_name=datetime.strftime(datetime.now(), '%d-%m_%H-%M')
os.mkdir('/content/drive/MyDrive/protein_classifier/{}'.format(folder_name))

earlystoppingcb=keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint=keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/protein_classifier/{}/checkpoint.h5'.format(folder_name),save_best_only=True)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


In [ ]:
history=model.fit(X_train, t_train_0, epochs=1300,batch_size=64, callbacks=[earlystoppingcb, checkpoint], validation_data=(X_val, t_val_0))


Epoch 1/1300
1378/1378 [==============================] - 63s 23ms/step - loss: 0.6746 - accuracy: 0.5799 - val_loss: 0.6482 - val_accuracy: 0.6175
Epoch 2/1300
1378/1378 [==============================] - 31s 22ms/step - loss: 0.6318 - accuracy: 0.6430 - val_loss: 0.5820 - val_accuracy: 0.6801
Epoch 3/1300
1378/1378 [==============================] - 31s 22ms/step - loss: 0.6079 - accuracy: 0.6636 - val_loss: 0.6549 - val_accuracy: 0.6156
Epoch 4/1300
1378/1378 [==============================] - 31s 22ms/step - loss: 0.5574 - accuracy: 0.7106 - val_loss: 0.5806 - val_accuracy: 0.7190
Epoch 5/1300
1378/1378 [==============================] - 31s 22ms/step - loss: 0.4751 - accuracy: 0.7722 - val_loss: 0.3843 - val_accuracy: 0.8342
Epoch 6/1300
1378/1378 [==============================] - 30s 22ms/step - loss: 0.4112 - accuracy: 0.8110 - val_loss: 0.3882 - val_accuracy: 0.8420
Epoch 7/1300
1378/1378 [==============================] - 30s 22ms/step - loss: 0.3616 - accuracy: 0.8414 - val_

In [ ]:
%cd /content/tp_curso_ml/src/pickle/json_files

with open('codigo_numerico.json') as file:
  cn=json.load(file)
sec_juan="""MATETVELHKLKLAELKQECLARGLETKGIKQDLIHRLQAYLEEHAEEEANEEDVLGDET
EEEETKPIELPVKEEEPPEKTVDVAAEKKVVKITSEIPQTERMQKRAERFNVPVSLESKK
AARAARFGISSVPTKGLSSDNKPMVNLDKLKERAQRFGLNVSSISRKSEDDEKLKKRKER
FGIVTSSAGTGTTEDTEAKKRKRAERFGIA"""
# arr_sec_juan=np.array(sec
translator=np.vectorize(lambda aa: cn.get(aa))
cn.get("M")
query_juan=translator([letter for letter in sec_juan])
query_juan

/content/tp_curso_ml/src/pickle/json_files


TypeError: ignored

In [ ]:
[letter for letter in sec_juan ]

['M',
 'A',
 'T',
 'E',
 'T',
 'V',
 'E',
 'L',
 'H',
 'K',
 'L',
 'K',
 'L',
 'A',
 'E',
 'L',
 'K',
 'Q',
 'E',
 'C',
 'L',
 'A',
 'R',
 'G',
 'L',
 'E',
 'T',
 'K',
 'G',
 'I',
 'K',
 'Q',
 'D',
 'L',
 'I',
 'H',
 'R',
 'L',
 'Q',
 'A',
 'Y',
 'L',
 'E',
 'E',
 'H',
 'A',
 'E',
 'E',
 'E',
 'A',
 'N',
 'E',
 'E',
 'D',
 'V',
 'L',
 'G',
 'D',
 'E',
 'T',
 '\n',
 'E',
 'E',
 'E',
 'E',
 'T',
 'K',
 'P',
 'I',
 'E',
 'L',
 'P',
 'V',
 'K',
 'E',
 'E',
 'E',
 'P',
 'P',
 'E',
 'K',
 'T',
 'V',
 'D',
 'V',
 'A',
 'A',
 'E',
 'K',
 'K',
 'V',
 'V',
 'K',
 'I',
 'T',
 'S',
 'E',
 'I',
 'P',
 'Q',
 'T',
 'E',
 'R',
 'M',
 'Q',
 'K',
 'R',
 'A',
 'E',
 'R',
 'F',
 'N',
 'V',
 'P',
 'V',
 'S',
 'L',
 'E',
 'S',
 'K',
 'K',
 '\n',
 'A',
 'A',
 'R',
 'A',
 'A',
 'R',
 'F',
 'G',
 'I',
 'S',
 'S',
 'V',
 'P',
 'T',
 'K',
 'G',
 'L',
 'S',
 'S',
 'D',
 'N',
 'K',
 'P',
 'M',
 'V',
 'N',
 'L',
 'D',
 'K',
 'L',
 'K',
 'E',
 'R',
 'A',
 'Q',
 'R',
 'F',
 'G',
 'L',
 'N',
 'V',
 'S',
 'S',
 'I',
 '